In [ ]:
import pandas as pd
import pyautogui
import pyperclip
import logging
import tkinter as tk
from tkinter import ttk, messagebox, filedialog

pyautogui.PAUSE = 0.1
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

ABA_PLANILHA = "Produto Detalhado"
COORDENADAS = {
    "F4": (237, 77),
    "primeira_aba": (886, 675),
    "modificar_valor": (649, 889),
    "adicionar_info": (75, 106),
    "avancar": (919, 570),
    "segurado_contato": (518, 894),
}

caminho_planilha = ""

def carregar_planilha(caminho):
    try:
        with pd.ExcelFile(caminho) as xls:
            df = pd.read_excel(xls, sheet_name=ABA_PLANILHA, skiprows=3)
            df.rename(columns={"Unnamed: 1": "Veiculo", "Unnamed: 2": "Valor"}, inplace=True)
        return df
    except Exception as e:
        logging.error(f"Erro ao carregar planilha: {str(e)}")
        messagebox.showerror("Erro", f"Erro ao carregar a planilha: {str(e)}")
        return None

def definir_mensagem(valor):
    if valor == 130:
        return "#informação interna fornecimento # - FORNECIMENTO EM TRATATIVA# RECEBIDO AVIÃOZINHO DA LOJA - NAO CABE ALTERACAO DE MAO DE OBRA ##"
    return "#informação interna fornecimento # - FORNECIMENTO EM TRATATIVA# RECEBIDO AVIÃOZINHO DA LOJA - FEITA ALTERACAO DE MAO DE OBRA ##"

def automatizar_tarefas(valor, mensagem):
    try:
        pyperclip.copy(str(valor))
        pyautogui.click(*COORDENADAS['F4'])
        pyautogui.click(*COORDENADAS["modificar_valor"])
        pyautogui.hotkey("ctrl", "v")

        if not messagebox.askyesno("Confirmação", f"Confirmar o valor copiado: R${valor}?"):
            mostrar_feedback("Automação cancelada.", "orange")
            return

        pyperclip.copy(mensagem)
        pyautogui.click(*COORDENADAS['adicionar_info'])      
        pyautogui.hotkey("ctrl", "v")
        pyautogui.moveTo(*COORDENADAS['avancar'])
        pyautogui.press('enter')
        pyautogui.press('f2')
        pyautogui.click(*COORDENADAS['segurado_contato'])
        pyautogui.press('insert')
        pyperclip.copy("#informação interna fornecimento # CASO SEGURADO ENTRAR EM CONTATO, SOLICITAR AGENDAR SERVIÇO COM A LOJA - O.S LIBERADA #")
        pyautogui.hotkey("ctrl", "v")
        pyautogui.click(*COORDENADAS['primeira_aba'])

        mostrar_feedback("Automação concluída.", "#4CAF50")
    except Exception as e:
        logging.error(f"Erro na automação: {e}")
        messagebox.showerror("Erro", f"Ocorreu um erro durante a automação: {e}")

def mostrar_feedback(mensagem, cor):
    label_feedback.config(text=mensagem, foreground=cor)

def limpar_opcoes():
    for widget in frame_canvas.winfo_children():
        widget.destroy()
    canvas.yview_moveto(0)
    canvas.config(scrollregion=canvas.bbox("all"))


def selecionar_planilha():
    global caminho_planilha
    caminho_planilha = filedialog.askopenfilename(filetypes=[("Arquivos Excel", "*.xlsx")])
    if caminho_planilha:
        mostrar_feedback("Planilha carregada com sucesso!", "green")
        limpar_opcoes()
    else:
        mostrar_feedback("Nenhuma planilha selecionada.", "orange")

def buscar_veiculo(event=None):
    veiculo = entrada_veiculo.get().strip()
    if not veiculo:
        messagebox.showwarning("Aviso", "Por favor, insira o nome de um veículo.")
        return
    if not caminho_planilha:
        messagebox.showerror("Erro", "Por favor, selecione a planilha primeiro.")
        return
    df = carregar_planilha(caminho_planilha)
    if df is None:
        return
    limpar_opcoes()
    try:
        resultados = df[df["Veiculo"].str.contains(veiculo, na=False, case=False)]
        if resultados.empty:
            mostrar_feedback("Nenhum veículo encontrado.", "orange")
        elif len(resultados) == 1:
            selecionar_veiculo(resultados.iloc[0])
        else:
            exibir_opcoes(resultados)
    except KeyError:
        messagebox.showerror("Erro", "A coluna 'Veiculo' não foi encontrada na planilha.")

def exibir_opcoes(filtrados):
    for _, row in filtrados.iterrows():
        veiculo = row["Veiculo"]
        valor = round(row["Valor"], 0)
        texto_btn = f"{veiculo} - R${valor}"
        ttk.Button(frame_canvas, text=texto_btn, command=lambda r=row: selecionar_veiculo(r)).pack(pady=4, padx=5, fill="x", expand=True)
    canvas.update_idletasks() # Ensure all widgets are rendered before calculating scrollregion
    canvas.config(scrollregion=canvas.bbox("all"))

def selecionar_veiculo(row):
    veiculo = row["Veiculo"]
    valor = round(row["Valor"], 0)
    mensagem = definir_mensagem(valor)
    automatizar_tarefas(valor, mensagem)

root = tk.Tk()
root.title("Alteração de Mão de Obra Automática")
root.geometry("700x600")
root.configure(bg="white")

style = ttk.Style()
style.theme_use("clam")

style.configure("TFrame", background="#f5f5f5")
style.configure("TLabel", background="#f5f5f5", foreground="black", font=("Segoe UI", 10))
style.configure("TButton", background="#e0e0e0", foreground="black", font=("Segoe UI", 10), borderwidth=1, relief="solid")
style.map("TButton",
          background=[("active", "#d5d5d5")],
          foreground=[("active", "black")])
style.configure("TEntry", fieldbackground="white", foreground="black")
style.configure("TLabelframe", background="#f5f5f5", foreground="black")
style.configure("TLabelframe.Label", background="#f5f5f5", foreground="black")


frame_main = ttk.Frame(root, padding=20)
frame_main.pack(fill="both", expand=True)

ttk.Button(frame_main, text="Selecionar Planilha", command=selecionar_planilha).pack(pady=10)

label_feedback = tk.Label(frame_main, text="", font=("Arial", 10), bg="white", fg="gray")
label_feedback.pack(pady=5)

ttk.Label(frame_main, text="Digite o nome do veículo:").pack(anchor="w", pady=(10, 10))
entrada_veiculo = ttk.Entry(frame_main, width=45)
entrada_veiculo.pack(pady=8)
entrada_veiculo.bind("<Return>", buscar_veiculo)

ttk.Button(frame_main, text="Buscar Veículo", command=buscar_veiculo).pack(pady=10)


frame_opcoes = ttk.LabelFrame(frame_main, text="Resultados encontrados")
frame_opcoes.pack(fill="both", expand=True, pady=10)

canvas = tk.Canvas(frame_opcoes, bg="white", highlightthickness=0)
scrollbar = ttk.Scrollbar(frame_opcoes, orient="vertical", command=canvas.yview)
frame_canvas = ttk.Frame(canvas)

canvas_window_id = canvas.create_window((0, 0), window=frame_canvas, anchor="nw")
canvas.configure(yscrollcommand=scrollbar.set)

def on_canvas_configure(event):
    canvas_width = event.width
    canvas.itemconfig(canvas_window_id, width=canvas_width)
    canvas.config(scrollregion=canvas.bbox("all"))

canvas.bind("<Configure>", on_canvas_configure)


def _on_mousewheel(event):
    canvas.yview_scroll(int(-1 * (event.delta / 120)), "units")

canvas.bind_all("<MouseWheel>", _on_mousewheel)
canvas.pack(side="left", fill="both", expand=True)
scrollbar.pack(side="right", fill="y")


root.mainloop()